In [1]:
import re
import requests
import pandas as pd

from bs4 import BeautifulSoup

In [2]:
class MissingDict(dict):
    __missing__ = lambda self, key: key

map_values = {"Brighton and Hove Albion": "Brighton", "Huddersfield Town":"Huddersfield","Manchester United": "Manchester Utd", "Newcastle United": "Newcastle Utd", "Nottingham Forest":"Nott'ham Forest","Tottenham Hotspur": "Tottenham","Sheffield United":"Sheffield Utd", "West Ham United": "West Ham","West Bromwich Albion":"West Brom", "Wolverhampton Wanderers": "Wolves","Wycombe Wanderers":"Wycombe","Sheffield Wednesday":"Sheffield Weds","Queens Park Rangers":"QPR","Rotherham United":"Rotherham Utd","Preston North End":"Preston","Peterborough United":"P'borough Utd","Charlton Athletic":"Charlton Ath","Bolton Wanderers":"Bolton","Blackburn Rovers":"Blackburn"} 
mapping = MissingDict(**map_values)

In [3]:
def get_market_value(years, league, div, country):
    market_value_df = pd.DataFrame()
    for season in years:
        url = f'https://www.transfermarkt.co.uk/{league.lower()}/startseite/wettbewerb/{country}{div}/plus/?saison_id={season}'
        headers = {
            "User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36"
            }
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        teams = []
        list = soup.select('td[class="hauptlink no-border-links"]')
        for y in range(len(list)):
            team = list[y].text.strip()
            teams.append(team)
        df = pd.DataFrame(teams)
        df.columns = ['Team']
        df['Season'] = season
        mvalue = []
        value = soup.select('td[class="rechts"]')
        value = value[1::2]
        value = value[1:]
        for x in range(len(value)):
            mvalue.append(value[x].text.strip())
        df['Market Value'] = mvalue
        df['Team'] = df['Team'].str.replace('FC','')
        df['Team'] = df['Team'].str.replace('A ','')
        df['Team'] = df['Team'].str.replace('&','and')
        df['Team'] = df['Team'].map(mapping)
        df['Team'] = df['Team'].str.strip()
        df['Market Value'] = df['Market Value'].str.replace('.','')
        df_temp = df['Market Value'].str.strip('€').str.extract(r'(\d+)([bnm]+)')
        df_temp2 = df_temp[0] + df_temp[1].map({'bn': '0000000', 'm':'0000'})
        df.drop('Market Value', axis=1, inplace=True)
        df['Market Value'] = df_temp2
        market_value_df = pd.concat([market_value_df, df])
    return market_value_df


In [4]:
years = list(range(2023, 2016, -1))
league = 'premier-league'
div = '1'
country = 'GB'
market_value_df = get_market_value(years, league, div, country)

In [5]:
league = 'championship'
div = '2'
market_value_df2 = get_market_value(years, league, div, country)

In [6]:
market_value_df2

,Team,Season,Market Value
0,Southampton,2023,332480000
1,Leeds United,2023,244500000
2,Leicester City,2023,242850000
3,Hull City,2023,126830000
4,Middlesbrough,2023,107580000
...,...,...,...
19,Preston,2017,24750000
20,Sheffield Utd,2017,23750000
21,Barnsley,2017,21700000
22,Millwall,2017,15130000


In [7]:
market_value_df = pd.concat([market_value_df, market_value_df2])

In [8]:
def get_market_value_current(season, league, div, country):
    url = f"https://www.transfermarkt.co.uk/{league.lower()}/marktwerteverein/wettbewerb/{country}{div}"#/plus/?stichtag={season}-0{month}-0{day}"
    headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36"}
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    teams = []
    list = soup.select('td[class="hauptlink no-border-links"]')
    teams = []
    for y in range(len(list)):
            team = list[y].text.strip()
            teams.append(team)
    df = pd.DataFrame(teams)
    df.columns = ['Team']
    df['Season'] = season
    mvalue = []
    value = soup.select('td[class="rechts hauptlink"]')
    value = value[:-10]
    for x in range(len(value)):
            mvalue.append(value[x].text.strip())
    df['Market Value'] = mvalue
    df['Team'] = df['Team'].str.replace('FC','')
    df['Team'] = df['Team'].str.replace('A ','')
    df['Team'] = df['Team'].str.replace('&','and')
    df['Team'] = df['Team'].map(mapping)
    df['Team'] = df['Team'].str.strip()
    df['Market Value'] = df['Market Value'].str.replace('.','')
    df_temp = df['Market Value'].str.strip('€').str.extract(r'(\d+)([bnm]+)')
    df_temp2 = df_temp[0] + df_temp[1].map({'bn': '0000000', 'm':'0000'})
    df.drop('Market Value', axis=1, inplace=True)
    df['Market Value'] = df_temp2
    return df

In [9]:
market_value_df_current = get_market_value_current(2024, 'premier-league', '1', 'GB')

In [10]:
market_value_df = pd.concat([market_value_df, market_value_df_current])

In [13]:
market_value_df.to_excel(r"C:\Users\George\Documents\University\Masters\DISS\Datasets\MARKET VALUE DATA.xlsx")